If in need of troubleshooting getting this notebook:

<!--NAVIGATION-->
 [Week 4](../2020-03-05/04_python_intro.ipynb) 

# Post-mortem

Possible reasons for failure of installation of poetry for many users last week:
    
+ The `curl` command line tool was not installed and so the installation command failed.
+ The PATH was not setup properly. This seems unlikely as the installation modifies the PATH in the active terminal and modifies both the .bash_profile and the .profile files to make sure that terminals subsequently opened will have the installation directory (~/.poetry/bin) on the PATH
+ On Windows systems it could be that ~/.bashrc is the appropriate setup script for the user and so the aforementioned path modifications did not work.

Other than these reasons, I have not thought of a way in which the poetry installation could have failed. It clearly did though so we will not be using poetry today.

### Possible remnants of the poetry installation attempt

+ In attempting to find an alternative installation approach I attempted to use the python packaging tool pip, and the general purpose software packaging tool conda. If you wish to try and reverse these changes you can consider executing the following commands. Be sure to read the accompanying description/warnings.

**Undoing the installation that used the curl tool**: The installation seemed to work for many people but did not work after that. You will not need poetry for the rest of the course. You can run the following cell to uninstall poetry from your system:

In [ ]:
#!python get-poetry.py --uninstall -y

**Undoing the installation that used the pip**: This uninstall may not work as well as the previous case. For you to be able to uninstall poetry, you need to know what pip you used to do this. It is likely the version of pip associated with the environment you run your notebooks with (though it may not be). The following command should uninstall poetry if it was installed via pip:

In [ ]:
#!pip uninstall poetry

**Resetting your base conda environment**: Assuming you have not created your own conda environments your only conda environment will be the base conda environment. Ideally we only use conda to install software into this environment. For various reasons we may not do this sometimes. As we continue to install software into conda environments they can start to get subtly broken and become difficult to install new software into them. One such example of this is that resolving dependencies takes a long time when we try to install some more software. There are [lots of ways](https://www.anaconda.com/understanding-and-improving-condas-performance/) to solve this last problem. Moving forward it is best practice to create new environments for new projects; creating an environment per project allows you to keep your dependencies concise and clean on a project basis and will prevent any dependencies issues when trying to resolve lots of dependencies. This is where conda really shines for ease and reproducibility.  
Ultimately for our purposes there is likely no reason why we don't just reset our base environment back to the default installation to avoid any of these issues. Assuming you have nothing that you wish to hang on to in the current environment you can reset it. 

**Note**: This will take a long time (likely 5-20 minutes) so you probably don't want to run it during class:

In [ ]:
#!conda install —revision 1

# Packaging with python

This lesson draws heavily on the [python guide to packaging](https://packaging.python.org/tutorials/packaging-projects/).



#### A very basic setup

We will start by creating a new directory for our work on a package:

In [ ]:
from class_setup import create_package_dir

create_package_dir("packaging_demo_in_class")

Now we will create a directory structure for our package:

In [ ]:
from pathlib import Path
package_name = "example_pkg"
Path('tests').mkdir()
python_dir = Path(package_name)
python_dir.mkdir()
(python_dir / '__init__.py').touch()
Path('setup.py').touch()
Path('LICENSE').touch()
Path('README.md').touch()

#### Adding metadata and installation details

We now have many of the files that should be in a basic package. Let's start to generate some of the details.

You can edit the following as you see fit. This setup.py file does the work for describing how your package is installed and telling users some of the details about package:

In [ ]:
%%writefile setup.py
import setuptools

with open("README.md", "r") as fh:
    long_description = fh.read()

setuptools.setup(
    name="example_pkg", 
    version="0.0.1",
    author="Example Author",
    author_email="author@example.com",
    description="A small example package",
    long_description=long_description,
    long_description_content_type="text/markdown",
    url="https://github.com/pypa/packaging_demo",
    packages=setuptools.find_packages(),
    classifiers=[
        "Programming Language :: Python :: 3",
        "License :: OSI Approved :: MIT License",
        "Operating System :: OS Independent",
    ],
    python_requires='>=3.6',

)

#### Describing our project to potential users


We should also always have a readme file to help our users to orient themselves. Since we would often use github to distribute our code, markdown is a sensible file format for this:

In [ ]:
%%writefile README.md
# Example Package

This is a simple example package. You can use
[Github-flavored Markdown](https://guides.github.com/features/mastering-markdown/)
to write your content.

#### Letting others use our code

You should always [choose a licence](https://choosealicense.com) to include with your code. It helps others to determine how they can use your code. Without a licence, most people simply cannot use your code based on their organizations regulations.

In [ ]:
%%writefile LICENSE
Copyright (c) 2018 The Python Packaging Authority

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

#### Add code to package up

In [ ]:
%%writefile example_pkg/calc.py
def add(x, y):
    """Add Function"""
    return x + y


def subtract(x, y):
    """Subtract Function"""
    return x - y


def multiply(x, y):
    """Multiply Function"""
    return x * y


def divide(x, y):
    """Divide Function"""
    if y == 0:
        raise ValueError('Can not divide by zero!')
    return x / y    

# see http://katyhuff.github.io/python-testing/03-exceptions/
def mean(num_list):
    try:
        return sum(num_list)/len(num_list)
    except ZeroDivisionError :
        return 0
    except TypeError as detail :
        msg = "The algebraic mean of an non-numerical list is undefined.\
               Please provide a list of numbers."
        raise TypeError(detail.__str__() + "\n" +  msg)


In [ ]:
%%writefile example_pkg/fizzbuzz.py
def fizzbuzz(n,fizz=3,buzz=5):
    output = []
    for number in range(n):
        if number % fizz == 0 and number % 5 == 0:
            output.append((number,"fizzbuzz"))
            continue
        elif number % 3 == 0:
            output.append((number,"fizz"))
            continue
        elif number % 5 == 0:
            output.append((number,"buzz"))
            continue
    return output

def main():
    print(fizzbuzz(10))


if __name__ == '__main__':
    main()

And we are done! We now have all of the files to install our project, and a little code to see what happens. There are 3 ways of installing this code:
    
1. Upload our code to PyPA. We won't do this. It's not hard but you will really want to share your code before you want to do that.
2. Install directly from github. As our package is now we are able to upload it to github and then have other users install our project directly from there!
3. Install it from our local version. This is the easiest for now so lets do that.

In [ ]:
pip install .

# Revisiting tests

Let's once again try to run or tests from last week. We'll copy the files from last week and then see if we can run them.

In [ ]:
for f in Path('../../2020-04-02/tests').glob('*.py'):
    (Path("tests") / f.name).write_text(f.read_text())